In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing, metrics 
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from keras.models import Model, Sequential
from keras.wrappers.scikit_learn import KerasRegressor
from keras.layers import Input, Dense, Activation

Using TensorFlow backend.


In [2]:
#NOTE: none of these fxns are meant for single point input, input should always be a pd.DataFrame with more
#than one row, not an unreasonable request since sklearn makes you reshape 1D arrays.


# Data cleaning section: 

def split_and_scale(df_train, n):
    """Splits training dataframe into predictors and properties to be predicted and returns them in 2 new dfs.
       This function assumes all of the predictors are grouped together on the right side of the df.
       df_train: training df
       n: number of properties to be predicted(number of outputs)"""
    properties, predictors = split(df_train, n)
    # COMMENT OUT THIS LINE IF YOU DONT WANT TO HAVE POLYNOMIAL TERMS IN YOUR TRAINING DATA
    # But note that accuracy is much better with this, but the model will have higher variance
    predictors_polynomial = polynomialize(predictors)
    predictors_scaled_polynomial, predictors_scaler_polynomial = scaling(predictors_polynomial)
    return properties, predictors_scaled_polynomial, predictors_scaler_polynomial 


def polynomialize(series):
    """Adds polynomial features to degree 3, including interaction features. 
    series: an input ndarray of floats to be polynomialized.
    This function returns a ndarray of all of the features specified above."""
    # Creating polynomial object
    poly = PolynomialFeatures(degree = 3)
    # Adding polynomial terms
    predictors_polynomial = poly.fit_transform(series)
    return predictors_polynomial

# Still in development, in case we want to add more terms that aren't polynomial
# def add_nonlinear_terms(df, n):
#     properties = df[df.columns[-n:]]
#     predictors = df[df.columns[:-n]]
#     i = np.arange(len(predictors.columns) * 4)
#     x = 0
#     for column in predictors.values:
#         predictors.assign(i[x]=column**2)
#         predictors.assign(column**3)
#         predictors.assign(np.exp(column))
#         predictors.assign(np.sign(column))
#     return properties, predictors


def split(df, n):
    """Takes an input pd.DataFrame and returns 2 ndarrays of the properties 
    and predictors."""
    properties = df[df.columns[-n:]].values
    predictors = df[df.columns[:-n]].values
    return properties, predictors


def scaling(df_train):
    """This function takes a pd.DataFrame, creates a sklearn.StandardScaler, scales the DataFrame,
       and returns the scaled data in a pd.DataFrame as well as the sklearn.StandardScaler object
       for transforming data back to unscaled form post machine learning.
       df_train: pd.DataFrame(for our purposes should be of shape 20 columns by an arbitrary number of rows)"""
    #Creating scaler object
    scaler = preprocessing.MinMaxScaler()
    #Scaling df_train
    scaled_data = pd.DataFrame(scaler.fit_transform(df_train))
    
    return scaled_data, scaler

# Training/predicting


def train_model(df_train, df_validation, model, n):
    """This function takes a training DataFrame, validation DataFrame and a preconfigured model
       and trains said model on the training data followed by measuring error on validation data and 
       returning both the trained model and accuracy metric. This function assumes whatever parameter(s)
       being predicted is in the last column(s) of df_train.
       n: number of outputs
       because this function returns the trained model, more metrics can be performed later that are specific
       to whatever package it is in/the type of model it is
       Parameters"""
    #generating scaled data and their respective scaler objects
    t_properties, t_predictors_scaled, t_predictors_scaler = split_and_scale(df_train, n)
    v_properties, v_predictors_scaled, v_predictors_scaler = split_and_scale(df_validation, n)
    #supervised learning of predictors and properties to fit model, note: keras does not take pd.DataFrames for
    #training, using .values fixes this
    model.fit(t_predictors_scaled, t_properties)
    #predicting output of validation set
    predictions = pd.DataFrame(model.predict(v_predictors_scaled))
    #calculating RMSE from sklearn package
    val_error = np.sqrt(metrics.mean_squared_error(predictions, v_properties))
    return model, val_error, t_predictors_scaler


def model_prediction(test_data, fitted_model, scaler, n):
    """Takes a fitted model and predicts the output of test data, returns the predicted data and accuracy.
       THIS FUNCTION IS ONLY TO BE USED FOR FUTURE PREDICTIONS OR TESTING(WHICH SHOULD ONLY BE DONE ONCE).
       Do not use this while training a model, that's what the validation data will be used for. We do not 
       want to introduce bias into our model by fitting to the test data
       n = number of predictors"""
    
    #splitting predictors and properties
    properties, predictors = split(test_data, n)
    predictors = polynomialize(predictors)
    predictors_scaled = scaler.transform(predictors)
    #predicting based on scaled input predictors
    prediction = fitted_model.predict(predictors_scaled)
    #calculating MSE
    accuracy_metric = np.sqrt(metrics.mean_squared_error(properties, prediction))

    return prediction, accuracy_metric

# Below functions initialize all the different types of models we are looking at:


def neural_network():
    """Creates a neural network object to be passed into train_model function, can change properties of net
       here."""
    def model():
        model = Sequential()
        model.add(Dense(84, input_dim=84, kernel_initializer='normal', activation='relu'))
        model.add(Dense(84, kernel_initializer='normal', activation='relu'))
        model.add(Dense(1, kernel_initializer = 'normal'))#kernel_initializer = initial values of outputs i think
        model.compile(optimizer='adam', loss='mse')
        return model
    network = KerasRegressor(build_fn=model, epochs=100, batch_size=25000, verbose=0)
#     network.fit(x=None, y=None, batch_size=None, epochs=1, verbose=1, callbacks=None, validation_split=0.0, 
    return network


def linear_regression():
    """creates a linear regression object"""
    regr = LinearRegression()
    return regr



In [36]:
# to go in cleaning section
# import clean

def test_split():
    data = {'column1': [2, 2, 3], 'column2': [1, 3, 5]}
    df = pd.DataFrame(data)
    one, two = clean.split(df, 1)
    assert one[0] == 1
    assert two[0] == 2
    return

def test_scaling():
    data = {'column1': [2.0, 2.0, 3.0], 'column2': [1.0, 3.0, 5.0]}
    df = pd.DataFrame(data)
    df, scaler = clean.scaling(df)
    assert df.loc[0].iloc[0] == 0
    assert df.loc[2].iloc[0] == 1
    return



In [3]:
data = pd.read_csv('HCEPD_100K.csv')
data.head()

,id,SMILES_str,stoich_str,mass,pce,voc,jsc,e_homo_alpha,e_gap_alpha,e_lumo_alpha,tmp_smiles_str
0,655365,C1C=CC=C1c1cc2[se]c3c4occc4c4nsnc4c3c2cn1,C18H9N3OSSe,394.3151,5.161953,0.867601,91.567575,-5.467601,2.022944,-3.444656,C1=CC=C(C1)c1cc2[se]c3c4occc4c4nsnc4c3c2cn1
1,1245190,C1C=CC=C1c1cc2[se]c3c(ncc4ccccc34)c2c2=C[SiH2]...,C22H15NSeSi,400.4135,5.261398,0.504824,160.401549,-5.104824,1.630750,-3.474074,C1=CC=C(C1)c1cc2[se]c3c(ncc4ccccc34)c2c2=C[SiH...
2,21847,C1C=c2ccc3c4c[nH]cc4c4c5[SiH2]C(=Cc5oc4c3c2=C1...,C24H17NOSi,363.4903,0.000000,0.000000,197.474780,-4.539526,1.462158,-3.077368,C1=CC=C(C1)C1=Cc2oc3c(c2[SiH2]1)c1c[nH]cc1c1cc...
3,65553,[SiH2]1C=CC2=C1C=C([SiH2]2)C1=Cc2[se]ccc2[SiH2]1,C12H12SeSi3,319.4448,6.138294,0.630274,149.887545,-5.230274,1.682250,-3.548025,C1=CC2=C([SiH2]1)C=C([SiH2]2)C1=Cc2[se]ccc2[Si...
4,720918,C1C=c2c3ccsc3c3[se]c4cc(oc4c3c2=C1)C1=CC=CC1,C20H12OSSe,379.3398,1.991366,0.242119,126.581347,-4.842119,1.809439,-3.032680,C1=CC=C(C1)c1cc2[se]c3c4sccc4c4=CCC=c4c3c2o1


In [4]:
sample = data[['e_homo_alpha', 'jsc', 'e_gap_alpha', 'e_lumo_alpha', 'mass','voc', 'pce']]

In [5]:
# linear regression test - accuracy is 100x better after adding polynomial terms
model, val_error, scaler = train_model(sample.iloc[:80000], sample.iloc[80000:],linear_regression(), 1)
model_prediction(sample.iloc[0:5], model, scaler, 1)

(array([[5.16195320e+00],
        [5.26139772e+00],
        [5.32907052e-15],
        [6.13829370e+00],
        [1.99136566e+00]]), 5.02821956683809e-15)

In [6]:
# neural network test, make sure to update your layer for expected input
model1, val_error1, scaler1 = train_model(sample.iloc[:80000], sample.iloc[80000:],neural_network(), 1)

In [7]:
model_prediction(sample.iloc[0:5], model1, scaler1, 1)

(array([5.238229 , 5.182736 , 0.5232755, 6.0541496, 2.0207558],
       dtype=float32), 0.24239096160995363)

In [8]:
# TO REDUCE OVERFITTING: reduce degree of polynomial terms

In [9]:
#model might just not have the right things
#to reorder columns:
#cols = df.columns.tolist()
#df = df[cols] 

In [10]:
import numpy as np
import pandas as pd
from matminer.data_retrieval.retrieve_MP import MPDataRetrieval
mpdr = MPDataRetrieval(api_key='x5He3oeSg1eCaIU4')

In [11]:
def extract_xrd_data(xrd_data):
    """
    Extracts the relevant XRD data from the dictionary obtained from MPD
    
    Parameters:
    ----------
    xrd_data : Dictionary
      The dictionary of data obtained from MPD 
    
    Returns:
    ----------
    clean_df: Pandas dataframe
        The top 10 XRD peaks and their corresponding two theta values for the material
    """
    
    # Checking if data has 10 or more peaks
    # Will return nothing and skip the material if less than 10
    if len(xrd_data['Cu']['pattern']) >= 10:
        
        # Extracting out the amplitude and two theta values from the dictionary contained inside the received data
        # then turning it into a pandas dataframe.
        dirty_df = pd.DataFrame(xrd_data['Cu']['pattern'], columns=xrd_data['Cu']['meta']) # Converts data into dataframe
        dirty_df.drop(['hkl','d_spacing'], axis=1, inplace=True) # Disposes of the hkl and d-spacing data

        # Sorting the peaks into the top 10 with the highest peaks
        dirty_df.sort_values('amplitude', ascending=False, inplace=True) # Sorts peaks from highest to smallest
        dirty_df.reset_index(drop=True, inplace=True) # Reseting index
        clean_df = dirty_df[:10] # Dropping all peaks below the top ten 
        
        return clean_df
    
    else:
        return None

In [12]:
# Function to reformat the data after cleaning
# Takes the dataframe and turns it into a dictionary wwhere all data points have a unique key
def reformat_xrd_data(xrd_data):
    """
    Reformats the cleaned data obtained from the extract_xrd_data function into a dictionary
    
    Parameters:
    ----------
    xrd_data : Dictionary
      The dictionary of data obtained from MPD 
    
    Returns:
    ----------
    clean_df: Pandas dataframe
        The top 10 XRD peaks and their corresponding two theta values for the material
    """
    
    # Checks if data was returned from the extracted data function
    # Skips material if nothing is returned
    if isinstance(extract_xrd_data(xrd_data), pd.DataFrame):
        # Cleaning data and creating empty dictionary
        clean_df = extract_xrd_data(xrd_data)
        mat_dict = {}

        # Loop to assign each data point to a key and stores it within the dictionary
        for i in range(0,20):
            if i < 10:
                amp_key = ('amplitude_' + str(i))
                mat_dict[amp_key] = clean_df['amplitude'][i]

            else:
                theta_key = ('two_theta_' + str(i-10))
                mat_dict[theta_key] = clean_df['two_theta'][i-10]

        return mat_dict
    
    else:
        return None

In [15]:
# Function 
def produce_data(df):
    """
    Reformats the cleaned data obtained from the extract_xrd_data function into a dictionary
    
    Parameters:
    ----------
    xrd_data : Dictionary
      The dictionary of data obtained from MPD 
    
    Returns:
    ----------
    clean_df: Pandas dataframe
        The top 10 XRD peaks and their corresponding two theta values for the material
    """
    
    full_dict = {}
    
    for i in range(len(df)):
        
        if reformat_xrd_data(df['xrd'].iloc[i]) != None:
            ID = df.index[i]
            mat_dict = reformat_xrd_data(df['xrd'].iloc[i])
            full_dict[ID] = mat_dict
            
        else:
            continue
    
    full_df = pd.DataFrame.from_dict(full_dict, orient='index')
    
    return full_df

In [16]:
df = mpdr.get_dataframe(criteria='Si', properties=['xrd'])
produce_data(df).head()

,amplitude_0,amplitude_1,amplitude_2,amplitude_3,amplitude_4,amplitude_5,amplitude_6,amplitude_7,amplitude_8,amplitude_9,two_theta_0,two_theta_1,two_theta_2,two_theta_3,two_theta_4,two_theta_5,two_theta_6,two_theta_7,two_theta_8,two_theta_9
mp-1001113,100.0,78.288443,45.261343,40.440821,30.707488,26.515207,22.092113,21.737670,16.821364,16.690467,36.834948,46.091710,38.160738,171.874808,45.443514,176.442253,60.797841,148.648332,87.640251,80.839115
mp-1056579,100.0,91.828984,79.970463,45.423897,30.872127,26.312484,23.760237,15.234896,14.045885,13.052846,44.350136,167.464337,37.945623,73.006051,140.704562,54.746898,123.597176,85.886608,135.942882,93.269152
mp-10649,100.0,35.031568,24.901660,23.115411,22.516940,20.830194,16.146856,15.585780,15.106204,14.134877,43.934318,37.766028,130.152767,39.420530,72.471977,83.797856,55.734788,71.487094,121.359663,87.842079
mp-1072544,100.0,74.250236,32.864700,32.201109,18.813035,17.053640,15.411324,14.668014,14.527636,13.937115,173.362106,18.640121,46.743236,32.580631,160.193335,50.741974,54.524230,42.462094,26.480052,98.859603
mp-1079297,100.0,28.393003,27.930625,27.680183,27.341183,24.356947,23.828151,21.640950,21.245367,20.805431,27.389812,25.843727,50.463312,46.935059,171.425514,51.286901,20.161639,52.606616,17.884802,177.432037
